Upload Data to Azure Data Lake


In [ ]:
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import os

# Azure Storage Account credentials
account_name = "storio"
account_key = ""
container_name = "raw-data"

# Connect to the Azure Blob Storage
connection_string = f""
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# Ensure container exists
try:
    container_client = blob_service_client.create_container(container_name)
except Exception:
    print(f"Container '{container_name}' already exists.")

# Local dataset paths
local_files = {
    "sales.csv": "sales.csv",
    "customers.csv": "customers.csv",
    "marketing.csv": "marketing.csv"
}

# Upload each file
for file_name, local_path in local_files.items():
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=file_name)
    with open(local_path, "rb") as data:
        blob_client.upload_blob(data, overwrite=True)
    print(f"Uploaded {file_name} to Azure Data Lake.")

dbt Configuration

In [ ]:
import os
import subprocess

# Create dbt project directory
project_name = "retail_project"
os.makedirs(project_name, exist_ok=True)

# Create dbt project structure
dbt_project_content = f"""
name: '{project_name}'
version: '1.0'
config-version: 2
profile: '{project_name}'
"""
with open(f"{project_name}/dbt_project.yml", "w") as f:
    f.write(dbt_project_content)

# Create profiles.yml content for Snowflake
profiles_content = f"""
{project_name}:
  outputs:
    dev:
      type: snowflake
      account: Justin Uto-Dieu
      user: JDIO
      password: password
      role: ACCOUNTADMIN
      database: your_database
      warehouse: your_warehouse
      schema: analytics
  target: dev
"""
os.makedirs(f"{os.path.expanduser('~')}/.dbt", exist_ok=True)
with open(f"{os.path.expanduser('~')}/.dbt/profiles.yml", "w") as f:
    f.write(profiles_content)

# Initialize dbt project
subprocess.run(["dbt", "init", project_name])
print(f"Initialized dbt project: {project_name}")